In [1]:
import datetime as dt
import geopandas as gpd
import os
import pandas as pd

os.environ["CALITP_BQ_MAX_BYTES"] = str(130_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
from siuba import *

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
SELECTED_DATE = dt.date(2021, 10, 22)

# Is this the correct query to use though?
# Perhaps this isn't service date?
shapes = (tbl.gtfs_schedule_type2.shapes()
 >> filter(_.calitp_itp_id==182)
 >> filter(_.calitp_extracted_at <= SELECTED_DATE, 
           _.calitp_deleted_at >= SELECTED_DATE)
)

In [6]:
# May need to add in fact_daily_trips
# Does query below need to have a trip_key or stop_key?

In [4]:
stop_info_on_day = (
    # Trips table has shape_id
    tbl.views.gtfs_schedule_dim_trips()
    >> select(_.calitp_itp_id, _.calitp_url_number, _.shape_id, 
              _.trip_id)
    >> distinct()
    # Attach historical shape_id
    >> inner_join(_, shapes, 
                  ["calitp_itp_id", "calitp_url_number", "shape_id"])
    # dim_stop_times has the stop_id and stop_sequence
    >> inner_join(_, 
                  (tbl.views.gtfs_schedule_dim_stop_times()
                   >> select(_.calitp_itp_id, _.calitp_url_number, 
                             _.trip_id, _.stop_id, _.stop_sequence
                            )
                  ), on = ["calitp_itp_id", "calitp_url_number", "trip_id"]
                 )
    # dim_stops has the stop_id and lat/lon
    >> inner_join(_,
                  (tbl.views.gtfs_schedule_dim_stops() 
                   >> select(_.calitp_itp_id, _.calitp_url_number, 
                             _.stop_id, _.stop_lon, _.stop_lat)
                  ), on = ["calitp_itp_id", "calitp_url_number", "stop_id"]
     )
)

In [5]:
stop_info_on_day


/opt/conda/lib/python3.9/site-packages/siuba/sql/utils.py:52: SAWarning: Dialect bigquery:bigquery will not make use of SQL compilation caching as it does not set the 'supports_statement_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Dialect maintainers should seek to set this attribute to True after appropriate development and testing for SQLAlchemy 1.4 caching support.   Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  return self.connectable.execute(*args, **kwargs)


,calitp_itp_id,calitp_url_number,shape_id,trip_id,shape_dist_traveled,shape_pt_lat,calitp_hash,shape_pt_sequence,calitp_deleted_at,calitp_extracted_at,shape_pt_lon,stop_id,stop_sequence,stop_lon,stop_lat
0,182,0,6990001_SoFi,54133798-SoFi_Stadium_Ex,None,33.9309587385,Llsqa7SBFNn2kezKwOpatw==,10029,2022-02-20,2021-08-18,-118.343894599,30022,1,-118.343425,33.952123
1,182,0,6990001_SoFi,54133798-SoFi_Stadium_Ex,None,33.9309587385,Llsqa7SBFNn2kezKwOpatw==,10029,2022-02-20,2021-08-18,-118.343894599,2319,2,-118.352446,33.933451
2,182,0,6990001_SoFi,54133869-SoFi_Stadium_Express_1530,None,33.9309587385,Llsqa7SBFNn2kezKwOpatw==,10029,2022-02-20,2021-08-18,-118.343894599,2319,2,-118.352446,33.933451
3,182,0,6990001_SoFi,54133869-SoFi_Stadium_Express_1530,None,33.9309587385,Llsqa7SBFNn2kezKwOpatw==,10029,2022-02-20,2021-08-18,-118.343894599,30022,1,-118.343425,33.952123
4,182,0,6990001_SoFi,54133870-SoFi_Stadium_Express_1530,None,33.9309587385,Llsqa7SBFNn2kezKwOpatw==,10029,2022-02-20,2021-08-18,-118.343894599,2319,2,-118.352446,33.933451
